In [3]:
import os

In [1]:
%pwd

'd:\\My_Docs\\TeaSicknessPrediction-Project\\research'

In [4]:
os.chdir("../")
%pwd

'd:\\My_Docs\\TeaSicknessPrediction-Project'

In [26]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [27]:
from TeaSicknessClassification.constants import *
from TeaSicknessClassification.utils.common import read_yaml, create_directories


In [29]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [30]:
import os
import urllib.request as request
import zipfile
from TeaSicknessClassification import logger
from TeaSicknessClassification.utils.common import get_size

In [31]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [34]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-31 21:58:43,570: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-31 21:58:43,572: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-31 21:58:43,575: INFO: common: created directory at: artifacts]
[2024-12-31 21:58:43,575: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-31 22:03:07,874: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 779945155
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'
Content-Type: application/zip
ETag: "04b4b89ffa11401cc3e41443473b01d5ca754a79451be06282502faa13eb1f3f"
Last-Modified: Tue, 31 Dec 2024 16:28:44 GMT
Strict-Transport-Security: max-age=31557600
timing-allow-origin: https://github.com
X-Content-Type-Options: nosniff
X-Frame-Options: deny
x-github-tenant: 
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 643C:0869:5B4A30:870C79:67741BBC
Accept-Ranges: bytes
Date: Tue, 31 Dec 2024 16:29